In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tabulate import tabulate

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from utils.data import load_breast_cancer_kagglehub, standardize_fit_transform
from utils.internal_metrics import (
    silhouette_score,
    davies_bouldin_index,
    calinski_harabasz_index,
    wcss,
)
from utils.external_metrics import (
    adjusted_rand_index,
    normalized_mutual_info,
    purity_score,
)
from GMM import GMM
from PCA import PCA

In [ ]:
X, y, feature_names = load_breast_cancer_kagglehub()
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
print("Data shape:", Xs.shape)

# ------------------- PCA components -------------------
components_list = [2, 5, 10, 15, 20]
cov_types = ['full', 'tied', 'diag', 'spherical']
seed = 42

# Store results
results = {}

In [ ]:
for m in components_list:
    pca = PCA(n_components=m, random_state=seed)
    Z = pca.fit_transform(Xs)
    results[m] = {}

    for cov in cov_types:
        bic_list, aic_list, loglik_list, gmm_models = [], [], [], []

        # Test GMM with k=2 clusters
        gmm = GMM(n_components=2, covariance_type=cov, max_iter=200, random_state=seed)
        gmm.fit(Z)

        bic_list.append(gmm.bic(Z))
        aic_list.append(gmm.aic(Z))
        loglik_list.append(gmm.log_likelihoods_)
        gmm_models.append(gmm)

        # Predict
        y_pred = gmm.predict(Z)

        # Internal metrics
        sil = silhouette_score(Z, y_pred)
        dbi = davies_bouldin_index(Z, y_pred)
        ch = calinski_harabasz_index(Z, y_pred)
        wcss_val = wcss(Z, y_pred, gmm.means_)

        # External metrics
        ari = adjusted_rand_index(y, y_pred)
        nmi = normalized_mutual_info(y, y_pred)
        pur = purity_score(y, y_pred)

        # Store everything
        results[m][cov] = {
            'BIC': bic_list[0],
            'AIC': aic_list[0],
            'loglik': loglik_list[0],
            'silhouette': sil,
            'davies_bouldin': dbi,
            'calinski_harabasz': ch,
            'wcss': wcss_val,
            'ARI': ari,
            'NMI': nmi,
            'Purity': pur,
            'model': gmm
        }    

In [ ]:
best_cov_per_dim = {}

for m in components_list:
    
    # Collect ALL values for min/max per metric (across cov_types)
    sil_vals = [results[m][cov]["silhouette"] for cov in cov_types]
    ch_vals = [results[m][cov]["calinski_harabasz"] for cov in cov_types]
    db_vals = [results[m][cov]["davies_bouldin"] for cov in cov_types]
    loglik_vals = [results[m][cov]["loglik"][-1] for cov in cov_types]
    bic_vals = [results[m][cov]["BIC"] for cov in cov_types]

    # Min-max per metric
    min_sil, max_sil = min(sil_vals), max(sil_vals)
    min_ch, max_ch = min(ch_vals), max(ch_vals)
    min_db, max_db = min(db_vals), max(db_vals)
    min_loglik, max_loglik = min(loglik_vals), max(loglik_vals)
    min_bic, max_bic = min(bic_vals), max(bic_vals)

    denom_sil = max(max_sil - min_sil, 1e-8)
    denom_ch = max(max_ch - min_ch, 1e-8)
    denom_db = max(max_db - min_db, 1e-8)
    denom_loglik = max(max_loglik - min_loglik, 1e-8)
    denom_bic = max(max_bic - min_bic, 1e-8)

    best_cov = None
    best_score = -np.inf

    for cov in cov_types:
        stats = results[m][cov]

        # Normalize each to [0,1], higher = better
        norm_sil = (stats["silhouette"] - min_sil) / denom_sil
        norm_ch = (stats["calinski_harabasz"] - min_ch) / denom_ch
        norm_db = (max_db - stats["davies_bouldin"]) / denom_db  # Lower DBI better
        norm_loglik = (stats["loglik"][-1] - min_loglik) / denom_loglik
        norm_bic = (max_bic - stats["BIC"]) / denom_bic  # Lower BIC better

        # Combined: sum (or np.mean([...]) for average)
        combined_score = norm_sil + norm_ch + norm_db + norm_loglik + norm_bic

        if combined_score > best_score:
            best_score = combined_score
            best_cov = cov

    best_cov_per_dim[m] = {"best_covariance": best_cov, "combined_score": best_score}


# Prepare DataFrame
df_best_cov = pd.DataFrame(best_cov_per_dim).T
df_best_cov.index.name = "PCA_components"

# Round values for better display
df_best_cov = df_best_cov.round(4)
print(tabulate(df_best_cov.reset_index(), headers='keys', tablefmt='fancy_grid', showindex=False))


In [ ]:
rows = []
for m in components_list:
    for cov in cov_types:
        stats = results[m][cov]
        rows.append({
            'PCA_components': m,
            'Covariance': cov,
            'silhouette': stats['silhouette'],
            'davies_bouldin': stats['davies_bouldin'],
            'BIC': stats['BIC'],
            'ARI': stats['ARI']
        })

df_long = pd.DataFrame(rows)
df_long = df_long.sort_values(['PCA_components', 'Covariance']).reset_index(drop=True)
print(df_long)